<a href="https://colab.research.google.com/github/bjimenezTechnodomus/licitaciones/blob/main/clasificacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import gspread

import pandas as pd
import re

gc = gspread.service_account()

In [2]:
worksheet = gc.open('Total partidas').sheet1

rows = worksheet.get_all_values()

data = pd.DataFrame.from_records(rows)
data.columns = data.iloc[0]
data.drop(0, inplace=True)
display(data.info())
display(data.describe())
respaldo = data.copy(deep=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 197716 entries, 1 to 197716
Data columns (total 19 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   Id_Partida                197716 non-null  object
 1   Archivo_Origen            197716 non-null  object
 2   Consecutivo               197716 non-null  object
 3   Número del Procedimiento  197716 non-null  object
 4   Tipo de Procedimiento     197716 non-null  object
 5   Convocante                197716 non-null  object
 6   Par- tida                 197716 non-null  object
 7   Cantidad Mínima           197716 non-null  object
 8   Cantidad Máxima           197716 non-null  object
 9   Unidad                    197716 non-null  object
 10  Descripción               197716 non-null  object
 11  Descripción de la Clave   197716 non-null  object
 12  Observaciones             197716 non-null  object
 13  Grupo de productos        197716 non-null  object
 14  Esta

None

,Id_Partida,Archivo_Origen,Consecutivo,Número del Procedimiento,Tipo de Procedimiento,Convocante,Par- tida,Cantidad Mínima,Cantidad Máxima,Unidad,Descripción,Descripción de la Clave,Observaciones,Grupo de productos,Estaba equivocada,Cuadro Básico,Nos interesa,Razones,Grupo Productos
count,197716,197716,197716,197716,197716,197716,197716,197716,197716,197716,197716,197716,197716,197716,197716,197716,197716,197716,197716
unique,196155,98,11000,5826,6,542,4779,2262,7341,4826,144208,15,9,14,4,17500,2,50,44
top,,10OH0SuqpTiohriWJiBOBehIAMDsAqTLWviTXKrpIKIs,1,LA-051GYN047-E66-2021,Adjudicación Directa,IMSS-Coordinación de Abastecimiento y Equipami...,1,,1,,060.456.0631.00 GUANTES. DE NITRILO O POLIBUTA...,Material de Curación,,No incluye,No,,,,
freq,7,5845,100,4120,66382,7031,5044,157523,29906,37272,68,122483,195146,194547,196607,95261,178401,169282,194968


In [4]:
data = respaldo[['Id_Partida', 'Convocante', 'Descripción', 'Descripción de la Clave', 'Grupo de productos', 'Cuadro Básico', 'Nos interesa', 'Grupo Productos']]
data.columns = ['id', 'convocante', 'descripcion', 'clave', 'grupo_1', 'cuadro_basico', 'interes', 'grupo_2']
data['cuadro_2'] = data['descripcion'].str.extract(r'(\d{3}[\.\s-]\d{3}[\.\s-]\d{4})').replace(r'[\s-]', '.')
CATEGORIAS = ['grupo_1', 'clave', 'interes', 'grupo_2']
data['grupo_2'] = data['grupo_2'].str.lower()
data['descripcion'] = data['descripcion'].str.lower()
data['grupo_2'][data['grupo_2'] == 'película y químicos dental'] = 'pelicula y quimicos dental'
data['grupo_2'][data['grupo_2'] == 'retos ind. quimicos'] = 'indicadores quimicos'
data['grupo_2'][data['grupo_2'] == 'reveladoras y casetas análogas'] = 'filme plano'
data['grupo_2'][data['grupo_2'] == 'mastografía'] = 'mastografias'
data['grupo_2'][data['grupo_2'] == 'rayos x digital'] = 'dr'
data['grupo_2'][data['grupo_2'] == 'mesa de rayos x'] = 'dr'
data['grupo_2'][data['grupo_2'] == 'dental digital'] = 'dr'
data['grupo_2'][data['grupo_2'] == 'película dry'] = 'pelicula dry'
data['grupo_2'][data['grupo_2'] == 'automaticas'] = 'servicios profesionales'
data['grupo_2'][data['grupo_2'] == 'impresoras'] = 'pelicula dry'
data['grupo_2'][data['grupo_2'] == 'pacs/ris'] = 'dr'
data[CATEGORIAS] = data[CATEGORIAS].apply(lambda x: x.astype('category'), axis=0)
categorias_dumies = pd.get_dummies(data[CATEGORIAS])
grupo_tech = pd.get_dummies(data['grupo_1'])
data['grupo_2'].value_counts()

/home/mauricio/.local/lib/python3.9/site-packages/pandas/core/frame.py:3599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item_frame_value(key, value)
/home/mauricio/.local/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/home/mauricio/.local/lib/python3.9/site-packages/pandas/core/generic.py:8861: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

                                    194968
bolsa / rollo mixta papel             1034
indicadores biologicos                 341
indicadores quimicos                   262
vapor                                  171
plasma                                 166
mastografias                           143
lavado y cuidado de instrumental        96
filme plano                             74
pelicula dry                            58
bolsa / rollo tyvek                     56
envoltura material no tejido            53
dr                                      52
desinfectantes                          48
selladoras                              43
contenedores                            24
incubadoras                             23
licenciamiento                          16
lavadoras ultrasonicas                  13
bolsa / rollo material no tejido        11
lavadoras ultrasónicas                  10
pelicula y quimicos dental              10
cr                                       8
monitores g

# Modelo de clasificación con todos los datos

Para generar un modelo de clasificación de los datos de las partidas a partir de regresión logística.

In [5]:
y = data['grupo_2']
X = data.drop('grupo_2', axis=1)
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(X, y):
  X_train = X.iloc[train_index]
  X_test = X.iloc[test_index]
  y_train = y.iloc[train_index]
  y_test = y.iloc[test_index]


In [6]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

tokens = '[A-Za-z0-9]+(?=\\s+)'

X_train['descripcion'].fillna('', inplace=True)

vectorizer = CountVectorizer(token_pattern= tokens)
vectorizer.fit_transform(X_train['descripcion'])

pl = Pipeline([
    ('vec', CountVectorizer(token_pattern = tokens)),
    ('clf', OneVsRestClassifier(LogisticRegression(max_iter=200)))
])

pl.fit(X_train['descripcion'], y_train)

/home/mauricio/.local/lib/python3.9/site-packages/pandas/core/generic.py:6383: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/home/linuxbrew/.linuxbrew/opt/python@3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vec', CountVectorizer(token_pattern='[A-Za-z0-9]+(?=\\s+)')),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(max_iter=200)))])

In [70]:
import numpy as np

data2 = data.sort_values('grupo_2')
no_producto = data2.iloc[0:116984]
productos = data2.iloc[116985:]
drops = np.random.choice(no_producto.index, 100000, replace=False)
no_producto.drop(drops, inplace=True)
data2 = pd.concat([no_producto, productos])
X2 = data2.drop('grupo_2', axis=1)
y2 = data2['grupo_2']
X2['descripcion'].fillna('', inplace=True)
vec = CountVectorizer(token_pattern=tokens)
vec.fit_transform(X_train['descripcion'])
pl.fit(X2['descripcion'], y2)


/home/mauricio/.local/lib/python3.9/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Pipeline(steps=[('vec', CountVectorizer(token_pattern='[A-Za-z0-9]+(?=\\s+)')),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(max_iter=200)))])

In [74]:
data2[['descripcion', 'cuadro_2', 'grupo_2']].to_csv('datos_entrenamiento.csv', index=False)


# Clasificación con Spacy

Una prueba para mejorar el modelo usando los modelos preentrenados de spacy

In [ ]:
# Ejemplo usando Spacy para la clasificaión de las partidas
!pip install --upgrade spacy

In [8]:
import spacy
nlp = spacy.blank('es')
textcat = nlp.add_pipe('textcat')
labels = data['grupo_2'].unique()
for label in labels:
  textcat.add_label(label)

train_text = data['descripcion'].values
train_labels = [{'cats': {label: case == label for label in labels}} for case in data['grupo_2']]
train_data = list(zip(train_text, train_labels))
train_data[:3]

In [10]:
from spacy.util import minibatch
from spacy.training.example import Example 

spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

batches = minibatch(train_data, size=10)
for batch in batches:
  for text, labels in batch:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, labels)
    nlp.update([example], sgd=optimizer)

## Ejemplo de aplicación del modelado

Este es un ejemplo del uso del algoritmo en un archivo anterior de partidas.

In [23]:
ejemplo = gc.open_by_key('1icObUrZVWNNgpvMS4DkCd-cH47VwmwI2FLa_0TBys9Y')
ejemplo = ejemplo.get_worksheet(1)
rows = ejemplo.get_all_values()
ejemplo = pd.DataFrame.from_records(rows)
ejemplo.columns = ejemplo.iloc[0]
ejemplo.drop(0, inplace=True)
ejemplo['Descripción'] = ejemplo['Descripción'].str.lower()
display(ejemplo.describe())

,Número del Procedimiento,Tipo de Procedimiento,Convocante,Par- tida,Cantidad Mínima,Cantidad Máxima,Unidad,Descripción,Descripción de la Clave,Observaciones
count,2791,2791,2791,2791,2791,2791,2791,2791,2791,2791
unique,30,5,19,1774,105,602,115,2004,5,2
top,LP-919044992-I03-2022,Licitación,Servicios de Salud de Nuevo León - Servicios d...,1,,Min. Max.,PIEZA C/1,600040109 (material de curacion ren.1) abatele...,Material de Curación,
freq,1774,1941,1774,25,2472,388,1018,3,2322,2787


In [24]:
textos = ejemplo['Descripción']
docs = [nlp.tokenizer(texto) for texto in textos]
textcat = nlp.get_pipe('textcat')
scores = textcat.predict(docs)
predicted_labels = scores.argmax(axis=1)
predicted_labels = [textcat.labels[label] for label in predicted_labels]
ejemplo['prediccion'] = predicted_labels

# Los resultados de las predicciones fueron peores que con regresión lógistica de sklearn con un tiempo de entrenamiento de 75 min.

NameError: name 'nlp' is not defined

In [25]:
ejemplo['prediccion'] = pl.predict(ejemplo['Descripción'])
ejemplo

,Número del Procedimiento,Tipo de Procedimiento,Convocante,Par- tida,Cantidad Mínima,Cantidad Máxima,Unidad,Descripción,Descripción de la Clave,Observaciones,prediccion
1,LA-012NCZ001-E7-2022,Licitación,INP-Dirección de Administración #012NCZ001,1,12,30,PZA,060.040.0840 (material de curacion ren.1) aguj...,Material Médico [biopsia*],,
2,LA-012NCZ001-E7-2022,Licitación,INP-Dirección de Administración #012NCZ001,2,54,135,PZA,060.040.0303 (material de curacion ren.2) aguj...,Material de Curación,,
3,LA-012NCZ001-E7-2022,Licitación,INP-Dirección de Administración #012NCZ001,3,5,12,ENV,060.046.1396 (material de curacion ren.3) alam...,Material de Curación,,
4,LA-012NCZ001-E7-2022,Licitación,INP-Dirección de Administración #012NCZ001,4,5,12,ENV,060.066.0401 (material de curacion ren.4) anti...,Material de Curación,,
5,LA-012NCZ001-E7-2022,Licitación,INP-Dirección de Administración #012NCZ001,5,191,477,ENV,060.082.0054 (material de curacion ren.5) apli...,Material de Curación,,
...,...,...,...,...,...,...,...,...,...,...,...
2787,AA-051GYN085-E1-2022,Adjudicación Directa,ISSSTE-Hospital Regional Gral. Ignacio Zaragoz...,19,,4,CAJA C/12 PZS,608411955 sutura seda negra trenzada # 1-0 caj...,Material de Curación,,
2788,AA-051GYN085-E1-2022,Adjudicación Directa,ISSSTE-Hospital Regional Gral. Ignacio Zaragoz...,20,,4,CAJA C/12 PZS,608414371 sutura catgut cromico # 2-0 caja c/1...,Material de Curación,,
2789,AA-051GYN085-E1-2022,Adjudicación Directa,ISSSTE-Hospital Regional Gral. Ignacio Zaragoz...,21,,10,PIEZA,609530266 venda de goma (smarch) de 6 cm. pieza,Material de Curación,,
2790,AA-051GYN085-E1-2022,Adjudicación Directa,ISSSTE-Hospital Regional Gral. Ignacio Zaragoz...,22,,10,PIEZA,609530282 venda de goma (smarch) de 8 cm. pieza,Material de Curación,,


In [75]:
todas = pd.read_csv('partidas_solicitante2.csv')
todas['categoria'] = pl.predict(todas['descripcion'])

In [76]:
todas.to_csv('partidas_etiquetadas.csv', index=False)

# Procesado de los datos de Cantidades

In [ ]:
textos = [text for text in data['Cantidad Máxima'] if len(text) > 5]

rango_maxmin = r'(\d+) a (\d+)'
cantidad_unidad = r'(\d+) (\w+)'
minmax = r'min+max'
dos_palabras = r'([A-Z]+) +([A-Z]+)'
miles = r'(\d{1,3}),(\d{3})'

display(textos)

In [ ]:
display([re.match(cantidad_unidad, texto).group(2) for texto in textos if re.match(cantidad_unidad, texto)])

In [ ]:
# Funciones utilitarias

def num_text_to_int(text):
  nums = ['cero', 'uno', 'dos', 'tres', 'cuatro', 'cinco', 'seis', 'siete', 'ocho', 'nueve', 'diez']

In [ ]:
data['Cantidad Máxima'] = (data['Cantidad Máxima'].str.replace('\D', ''))
data['Cantidad Mínima'] = (data['Cantidad Mínima'].str.replace('\D', ''))
data['Cantidad Máxima'] = data['Cantidad Máxima'].astype(int)
data['Cantidad Mínima'] = data['Cantidad Mínima'].astype(int)

In [ ]:
for index, num in enumerate(data['Cantidad Máxima']):
  print('{} {}'.format(index, int(num)))